In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# ENIGMA Toolbox Tutorial

### -----------------------------------
###  1. Load ENIGMA datasets
### -----------------------------------

In [ ]:
# Load summary statistics

from enigmatoolbox.datasets import load_summary_stats

# Load summary statistics for ENIGMA-Epilepsy
sum_stats = load_summary_stats('epilepsy')

# Get case-control subcortical volume and cortical thickness tables
SV = sum_stats['SubVol_case_vs_controls_ltle']
CT = sum_stats['CortThick_case_vs_controls_ltle']

# Extract Cohen's d values
SV_d = SV['d_icv']
CT_d = CT['d_icv']

print(SV_d)

### --------------------------------------
###  2. Surface data visualization
### --------------------------------------

In [ ]:
from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical

# Map parcellated data to the surface
CT_d_fsa5 = parcel_to_surface(CT_d, 'aparc_fsa5')

# Project the results on the surface brain
plot_cortical(array_name=CT_d_fsa5, surface_name="fsa5", size=(1200, 200),
              cmap='RdBu_r', color_bar=True, color_range=(-0.5, 0.5),
              embed_nb=True, interactive=False)

In [ ]:
from enigmatoolbox.plotting import plot_subcortical

# Project the results on the surface brain
plot_subcortical(array_name=SV_d, size=(1200, 200), embed_nb=True, interactive=False,
                 cmap='RdBu_r', color_bar=True, color_range=(-0.5, 0.5))

### -----------------------------------------------
###  3. Economo-Koskinas stratification
### -----------------------------------------------

In [ ]:
from enigmatoolbox.histology import economo_koskinas_spider

# Stratify cortical atrophy based on Economo-Koskinas classes
class_mean = economo_koskinas_spider(CT_d, axis_range=(-0.4, 0))

### ------------------------------------
### 4. Hub susceptibility model
### ------------------------------------

In [ ]:
from enigmatoolbox.datasets import load_sc, load_fc
from nilearn import plotting

# Load cortico- and sucortico-cortical functional connectivity data
fc_ctx, fc_ctx_labels, fc_sctx, fc_sctx_labels = load_fc()

In [ ]:
import numpy as np
from enigmatoolbox.plotting import plot_cortical
from enigmatoolbox.utils.parcellation import parcel_to_surface

# Compute weighted degree centrality measures from the connectivity data
fc_ctx_dc = np.sum(fc_ctx, axis=0)

# Map parcellated data to the surface
fc_ctx_dc_fsa5 = parcel_to_surface(fc_ctx_dc, 'aparc_fsa5')

# Project the results on the surface brain
plot_cortical(array_name=fc_ctx_dc_fsa5, surface_name="fsa5", size=(1200, 200),
              embed_nb=True, cmap='Reds', color_bar=True, color_range=(20, 30),
              interactive=False)

In [ ]:
import numpy as np
from enigmatoolbox.plotting import plot_subcortical

# Compute weighted degree centrality measures from the connectivity data
fc_sctx_dc = np.sum(fc_sctx, axis=1)

# Project the results on the surface brain
plot_subcortical(array_name=fc_sctx_dc, ventricles=False, size=(1200, 200),
                 cmap='Reds', color_bar=True, color_range=(5, 10),
                 embed_nb=True, interactive=False)

In [ ]:
import numpy as np

# Remove subcortical values corresponding to the ventricles
# (as we don't have connectivity values for them!)
SV_d_noVent = SV_d.drop([np.where(SV['Structure'] == 'LLatVent')[0][0],
                        np.where(SV['Structure'] == 'RLatVent')[0][0]]).reset_index(drop=True)


# Perform spatial correlations between functional hubs and Cohen's d
fc_ctx_r = np.corrcoef(fc_ctx_dc, CT_d)[0, 1]
fc_sctx_r = np.corrcoef(fc_sctx_dc, SV_d_noVent)[0, 1]

# Store correlation coefficients
rvals = {'functional cortical hubs': fc_ctx_r, 'functional subcortical hubs': fc_sctx_r}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Store degree centrality and atrophy measures
meas = {('functional cortical hubs', 'cortical thickness'): [fc_ctx_dc, CT_d],
        ('functional subcortical hubs', 'subcortical volume'): [fc_sctx_dc, SV_d_noVent]}

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

for k, (fn, dd) in enumerate(meas.items()):
    # Plot relationships between hubs and atrophy
    axs[k].scatter(meas[fn][0], meas[fn][1], color='#A8221C')
    m, b = np.polyfit(meas[fn][0], meas[fn][1], 1)
    axs[k].plot(meas[fn][0], m * meas[fn][0] + b, color='#A8221C')
    axs[k].set_ylim((-1, 0.5))
    axs[k].set_xlabel('{}'.format(fn[0].capitalize()))
    axs[k].set_ylabel('{}'.format(fn[1].capitalize()))
    axs[k].spines['top'].set_visible(False)
    axs[k].spines['right'].set_visible(False)

fig.tight_layout()
plt.show()

### ---------------------------------
### 5. Spin permutation tests
### ---------------------------------

In [ ]:
from enigmatoolbox.permutation_testing import spin_test

# Spin permutation testing for two cortical maps
fc_ctx_p, fc_ctx_d = spin_test(fc_ctx_dc, CT_d, surface_name='fsa5', parcellation_name='aparc',
                               type='pearson', n_rot=1000, null_dist=True)

# Store p-values and null distributions
p_and_d = {'functional cortical hubs': [fc_ctx_p, fc_ctx_d]}

In [ ]:
import matplotlib.pyplot as plt

# Plot null distributions
fig, axs = plt.subplots(1, figsize=(10, 5))

for k, (fn, dd) in enumerate(p_and_d.items()):   
    axs.hist(dd[1], bins=50, density=True, color='#A8221C', edgecolor='white', lw=0.5)
    axs.axvline(rvals['functional cortical hubs'], lw=1.5, ls='--', color='k', dashes=(2, 3),
                   label='$r$={:.2f}'.format(rvals['functional cortical hubs']) + '\n$p$={:.3f}'.format(dd[0]))
    axs.set_xlabel('Null correlations \n ({})'.format('functional cortical hubs'))
    axs.set_ylabel('Density')
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.legend(loc=1, frameon=False)

fig.tight_layout()
plt.show()